In [1]:
import sqlite3

conn = sqlite3.connect('sanfrancisco.db')

In [2]:
c = conn.cursor()
c.execute("SELECT Sold_Price, Bedroom, Bathroom, Sq_ft, Zipcode, Original_Price FROM Address WHERE Sq_ft is not null and Sold_Price>0")
rows = c.fetchall()

In [3]:
import random
# Didn't want the data to be biased by date
random.shuffle(rows)

y = []
X = []

for row in rows:
    
    if row[5] is None:
        continue
#Normalizing    
    sold_price = row[0] / 1000000.0
    bedroom = row[1]
    bathroom = row[2]
    sq_ft = row[3] / 1000.0
#     zip_code = row[4] - 94100
    original_price = row[5] / 1000000.0
    
    y.append(sold_price)
    X.append([bedroom, bathroom, sq_ft, original_price])

In [4]:
from math import floor

# y = y[:230]
# X = X[:230]
# Training a ratio of 70% if we didn't do the shuffle 
# my information would be biased towards old listings without taking new listings into account
training_ratio = .7
training_count = floor(len(y) * training_ratio)
testing_count = len(y) - training_count

print(len(y))
print(training_count)
print(testing_count)
print(testing_count + training_count)

352
246
106
352


In [5]:
training_y = y[0:training_count]
training_X = X[0:training_count]

testing_y = y[len(rows)-testing_count:]
testing_X = X[len(rows) - testing_count:]

In [6]:
from sklearn import linear_model as lm

reg = lm.LinearRegression()
reg.fit(training_X, training_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
reg.coef_
#Bedroom,Bathroom,Square Ft. 21.,Listing Price

array([-0.08766312,  0.08873556, -0.03656693,  0.92095353])

In [8]:
# reg.predict([[4, 3.5, 2.75]])

In [9]:
predictions = reg.predict(testing_X)

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(testing_y, predictions)
r2 = r2_score(testing_y, predictions)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R2 ): {r2}")

Mean Squared Error (MSE): 0.08969494001088454
R-squared (R2 ): 0.9820689268218238


In [11]:
reg.score(testing_X, testing_y)

0.9820689268218238

In [12]:
# 33 Dellbrook Ave
reg.predict([[4, 3, 1.911, 1.495]]) # Redfin estimate $1.710 mil

array([1.68208705])

In [13]:
# 1430 23rd Ave
reg.predict([[3,2.5,2.025,1.598]]) #Redfin Est. 1.896

array([1.82181381])

In [14]:
reg.predict([[3, 3, 2.102, 1.599]])

array([1.84398567])

In [15]:
reg.predict([[4, 2.5, 2.350, 1.498]])

array([1.66360927])

In [16]:
import pickle

filename = "model_v2.sav"
pickle.dump(reg, open(filename, 'wb'))